In [ ]:
## GPT Architecture
import math
import tiktoken

import torch
import torch.nn as nn
import torch.nn.functional as F

CONFIG = {
    'vocab_size': 50257,
    'emb_dim': 768,
    'context_length': 1024,
    'n_heads': 12,
    'n_layers': 12,
    'drop_rate': 0.1,
    'qkv_bias': False
}

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, num_heads, context_length=1024, dropout=0.0, qkv_bias=False):
        super().__init__()

        assert (d_out % num_heads) == 0, "d_out should be a multiple of num_heads"


        self.dk = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        self.Wq = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.Wk = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.Wv = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.dropout = nn.Dropout(dropout)
        self.output = nn.Linear(d_out, d_out, bias=qkv_bias)

        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, X):
        batch_size, token_size, embedding_dim = X.shape

        Q = self.Wq(X)
        K = self.Wk(X)
        V = self.Wv(X)

        Q = Q.view(batch_size, token_size, self.num_heads, self.head_dim)
        K = K.view(batch_size, token_size, self.num_heads, self.head_dim)
        V = V.view(batch_size, token_size, self.num_heads, self.head_dim)

        Q = Q.transpose(1, 2)
        K = K.transpose(1, 2)
        V = V.transpose(1, 2)

        raw_attention_scores = Q @ K.transpose(2, 3) / math.sqrt(K.shape[-1])
        masked_attention_scores = torch.masked_fill(
            raw_attention_scores, self.mask.bool()[:token_size, :token_size], -math.inf
        )
        attention_scores = self.dropout(F.softmax(masked_attention_scores, dim=-1))
        context_vectors = attention_scores @ V
        context_vectors = context_vectors.transpose(1, 2)
        context_vectors = context_vectors.contiguous().view(batch_size, token_size, self.dk)
        context_vectors = self.output(context_vectors)
        return context_vectors

class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()

        self.tok_emb = nn.Embedding(cfg['vocab_size'], cfg['emb_dim'])
        self.pos_emb = nn.Embedding(cfg['context_length'], cfg['emb_dim'])
        self.drop_emb = nn.Dropout(cfg['drop_rate'])

        self.trf_blocks = nn.Sequential(*[TransformerBlock(cfg) for _ in range(cfg['n_layers'])])
        self.final_norm = LayerNorm(cfg['emb_dim'])
        self.out_head = nn.Linear(cfg['emb_dim'], cfg['vocab_size'], bias = False)


    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_emb = self.tok_emb(in_idx)
        pos_emb = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_emb + pos_emb
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits
    
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean)/torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + math.tanh(2.0/torch.pi)) * (x + 0.044715 * torch.pow(x, 3))
    
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg['emb_dim'], 4 * cfg['emb_dim']),
            GELU(),
            nn.Linear(4 * cfg['emb_dim'], cfg['emb_dim']),
        )
    
    def forward(self, X):
        return self.layers(X)
    
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()

        self.att = MultiHeadAttention(
            d_in = cfg['emb_dim'],
            d_out = cfg['emb_dim'],
            context_length=cfg['context_length'],
            num_heads = cfg['n_heads'],
            dropout = cfg['drop_rate'],
            qkv_bias = cfg['qkv_bias']
        )

        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg['emb_dim'])
        self.norm2 = LayerNorm(cfg['emb_dim'])
        self.drop_shortcut = nn.Dropout(cfg['drop_rate'])

    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        return x

    

tokenizer = tiktoken.get_encoding('gpt2')
batch = []
txt1 = "Every effort moves you"
txt2 = "Every day holds a"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim = 0)
print(batch)

model = GPTModel(CONFIG)
output = model(batch)
output.shape

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


torch.Size([2, 4, 50257])

In [17]:
torch.manual_seed(123)
x = torch.rand(2, 4, 768)
block = TransformerBlock(CONFIG)
output = block(x)
x.shape, output.shape

(torch.Size([2, 4, 768]), torch.Size([2, 4, 768]))

In [30]:
def detailed_parameter_count(model):
    total = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            num_params = param.numel()
            print(f"{name}: {num_params:,} parameters")
            # print(f"{num_params:,}")
            total += num_params
    print(f"Total trainable: {total:,}")
    return total

detailed_parameter_count(model)

tok_emb.weight: 38,597,376 parameters
pos_emb.weight: 786,432 parameters
trf_blocks.0.att.Wq.weight: 589,824 parameters
trf_blocks.0.att.Wk.weight: 589,824 parameters
trf_blocks.0.att.Wv.weight: 589,824 parameters
trf_blocks.0.att.output.weight: 589,824 parameters
trf_blocks.0.ff.layers.0.weight: 2,359,296 parameters
trf_blocks.0.ff.layers.0.bias: 3,072 parameters
trf_blocks.0.ff.layers.2.weight: 2,359,296 parameters
trf_blocks.0.ff.layers.2.bias: 768 parameters
trf_blocks.0.norm1.scale: 768 parameters
trf_blocks.0.norm1.shift: 768 parameters
trf_blocks.0.norm2.scale: 768 parameters
trf_blocks.0.norm2.shift: 768 parameters
trf_blocks.1.att.Wq.weight: 589,824 parameters
trf_blocks.1.att.Wk.weight: 589,824 parameters
trf_blocks.1.att.Wv.weight: 589,824 parameters
trf_blocks.1.att.output.weight: 589,824 parameters
trf_blocks.1.ff.layers.0.weight: 2,359,296 parameters
trf_blocks.1.ff.layers.0.bias: 3,072 parameters
trf_blocks.1.ff.layers.2.weight: 2,359,296 parameters
trf_blocks.1.ff.layer

163000320